In [161]:
import glob
import pandas as pd
from datetime import datetime
import requests
import json

In [162]:
working_path = r"C:\Users\r2d2\Desktop\Data Engineering\coursera_sources\5_excercise - Peer Review  Assignment"

In [163]:
# Using source data from local machine
r1 = requests.get("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json")
r2 = requests.get("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json")
csv_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv"

In [164]:
bank_market_cap_1_json = r1.json()
bank_market_cap_2_json = r2.json()
exchange_rates_csv = pd.read_csv(csv_url)

with open(working_path + "/bank_market_cap_1.json", "w") as f:
    json.dump(bank_market_cap_1_json, f)
    
with open(working_path + "/bank_market_cap_2.json", "w") as f:
    json.dump(bank_market_cap_2_json, f)

# Excercise 1 - create Extract JSON function

In [165]:
def extract_json(file):
    dataframe = pd.read_json(file)
    return dataframe

In [166]:
def extract():
    extracted_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)'])
    
    for json_file in glob.glob(r"C:\Users\r2d2\Desktop\Data Engineering\coursera_sources\5_excercise - Peer Review  Assignment\*.json"):
        extracted_data = extracted_data.append(extract_json(json_file), ignore_index=True)
        
    return extracted_data

# Excercise 2

In [167]:
extracted_data = extract()
extracted_data

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.560


In [168]:
exchange_rates_csv = pd.read_csv(csv_url,index_col=0) # index_col=0 to not user first index column
# exchange_rates_csv.set_axis(["Country","Rates"], axis=1, inplace=True) # to work delete index_col=0
# exchange_rate = exchange_rates_csv.loc[exchange_rates_csv["Country"]=="GBP"] #withous index_col=0 u can select GBP like this
exchange_rates_csv

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083
CNY,6.483139
CZK,21.510117
DKK,6.119757
EUR,0.822504
GBP,0.732398


# Select GBP row

In [169]:
exchange_rate = exchange_rates_csv.loc["GBP"]

In [170]:
exchange_rate

Rates    0.732398
Name: GBP, dtype: float64

# Excercise 3

In [171]:
def transform(data):
    data["Market Cap (US$ Billion)"] = round(data["Market Cap (US$ Billion)"]*exchange_rate[0], 3)
    data.rename(columns = {"Market Cap (US$ Billion)":"Market Cap (GBP$ Billion)"}, inplace=True)
    return data

In [172]:
transformed_data = transform(extracted_data)

In [173]:
transformed_data

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519
...,...,...
100,Ping An Bank,27.826
101,Standard Chartered,27.332
102,United Overseas Bank,25.728
103,QNB Group,24.579


# Excercise 4 

In [174]:
def load(target_file, data_to_load):
    data_to_load.to_csv(target_file, index=False)

# Excercise 5

In [175]:
def log(message):
    timestamp_format = "%Y-%h-%d-%H:%M:%S"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(working_path + '/logging.txt', "a") as f:
        f.write(timestamp + "," + message + "\n")

In [176]:
extracted_data = extract()
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [177]:
log("Extract phase Ended")

In [178]:
# Write your code here
log("Transform phase Started")

In [179]:
transformed_data = transform(extracted_data)
transfomed_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [180]:

log("Load phase Started")

In [181]:

target_file = r"C:\Users\r2d2\Desktop\Data Engineering\coursera_sources\5_excercise - Peer Review  Assignment\final.csv"
load(target_file, transformed_data)

In [182]:
log("Load phase Ended")